In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import tushare as ts
import QUANTAXIS as QA
import talib as ta
import datetime
# import fix_yahoo_finance as yf

import pandas_datareader.data as web
from tqdm import tqdm_notebook

# yf.pdr_override() #需要调用这个函数

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [3]:
symbol_list = pd.read_excel('data/etf_pair_code.xlsx', 1, dtype={'symbol':str})

In [4]:
symbol_list[symbol_list.region == 'HK Equity'].head()

,symbol,region
160,700,HK Equity
163,992,HK Equity
164,2382,HK Equity
165,981,HK Equity
166,2018,HK Equity


In [5]:
symbol_list.region.value_counts()

US Equity     817
HK Equity      54
JP Equity      17
KS Equity       8
TT Equity       6
SS Equity       6
LN Equity       6
FH Equity       2
CN Equity       2
NO Equity       2
NA Equity       1
GR Equity       1
ita             1
HKD Curncy      1
FP Equity       1
Name: region, dtype: int64

In [12]:
symbol_list.query("region=='HK Equity'").head(20)

,symbol,region
160,700,HK Equity
163,992,HK Equity
164,2382,HK Equity
165,981,HK Equity
166,2018,HK Equity
167,522,HK Equity
168,696,HK Equity
169,148,HK Equity
170,268,HK Equity
171,3396,HK Equity


In [4]:
hk_symbol_list = symbol_list.query("region=='HK Equity'").symbol
hk_symbol_list = hk_symbol_list.apply(lambda x : x.zfill(4)+'.hk')
hk_symbol_list.head()

160    0700.hk
163    0992.hk
164    2382.hk
165    0981.hk
166    2018.hk
Name: symbol, dtype: object

In [4]:
from time import sleep
from tqdm import tqdm

In [8]:
from tqdm import tnrange
for i in tqdm_notebook(range(10), desc='1st loop'):
    for j in tnrange(100, desc='2nd loop', leave=False):
        sleep(0.01)

In [6]:
hk_data = pd.DataFrame()
start = '2010-01-01'
end = '2018-12-21'

for i in tqdm_notebook(hk_symbol_list):
    try:
        temp_data = web.DataReader(i, 'yahoo', start, end)
        if len(temp_data) == 0:
            print('could not retrieve the %s data' % i)
        temp_data = temp_data.assign(code=i)
        hk_data = pd.concat([hk_data, temp_data])
    except:
        print('there is something wrong in %s' % i)
    

could not retrieve the 0700.hk data
could not retrieve the 0992.hk data
could not retrieve the 2382.hk data
could not retrieve the 0981.hk data
could not retrieve the 2018.hk data
could not retrieve the 0522.hk data
could not retrieve the 0696.hk data
could not retrieve the 0148.hk data
could not retrieve the 0268.hk data
could not retrieve the 3888.hk data
could not retrieve the 0763.hk data
could not retrieve the 0354.hk data
could not retrieve the 0285.hk data
could not retrieve the 3800.hk data
could not retrieve the 1888.hk data
could not retrieve the 8083.hk data
could not retrieve the 0582.hk data
could not retrieve the 0861.hk data
could not retrieve the 2662.hk data
could not retrieve the 0777.hk data
could not retrieve the 0698.hk data
could not retrieve the 1686.hk data
could not retrieve the 2280.hk data
could not retrieve the 1385.hk data
could not retrieve the 0732.hk data
could not retrieve the 0818.hk data
could not retrieve the 3336.hk data
could not retrieve the 3393.

In [7]:
hk_data.head()

,High,Low,Open,Close,Volume,Adj Close,code
Date,,,,,,,
2015-06-29,43.549999,42.849998,43.150002,42.950001,62651090.0,41.258984,3396.hk
2015-06-30,43.450001,43.000000,43.000000,43.349998,8230600.0,41.643234,3396.hk
2015-07-02,43.950001,42.950001,43.349998,43.849998,7664889.0,42.123547,3396.hk
2015-07-03,43.900002,42.950001,43.849998,43.000000,5044400.0,41.307014,3396.hk
2015-07-06,43.500000,40.900002,43.049999,42.549999,29058194.0,40.874729,3396.hk


In [40]:
quandl_key = 'sCpERv7Umk1JuwpSfgE5'
web.DataReader('MSFT.US', 'quandl', '2018-04-01', '2018-12-31', 
               access_key=quandl_key).head()

,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Date,,,,,,,,,,,,


In [33]:
ALPHAVANTAGE_API_KEY = '3F73H1FLD4EDDNSJ'
web.DataReader('AAPL', 'av-daily-adjusted', start, end, 
               access_key=ALPHAVANTAGE_API_KEY).tail()

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient
2018-12-17,165.45,168.35,162.73,163.94,163.94,44287922,0.0,1.0
2018-12-18,165.38,167.53,164.39,166.07,166.07,33841518,0.0,1.0
2018-12-19,166.00,167.45,159.09,160.89,160.89,49047297,0.0,1.0
2018-12-20,160.40,162.11,155.30,156.83,156.83,64772960,0.0,1.0
2018-12-21,156.86,158.16,149.63,150.73,150.73,95744384,0.0,1.0


In [44]:
web.DataReader('XYZ', 'fred', start, end).tail()

RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=XYZ

In [6]:
hk_data.head()

,High,Low,Open,Close,Volume,Adj Close,code
Date,,,,,,,
2015-06-29,43.549999,42.849998,43.150002,42.950001,62651090.0,41.258984,3396.hk
2015-06-30,43.450001,43.000000,43.000000,43.349998,8230600.0,41.643234,3396.hk
2015-07-02,43.950001,42.950001,43.349998,43.849998,7664889.0,42.123547,3396.hk
2015-07-03,43.900002,42.950001,43.849998,43.000000,5044400.0,41.307014,3396.hk
2015-07-06,43.500000,40.900002,43.049999,42.549999,29058194.0,40.874729,3396.hk


In [11]:
hk_symbol_list.head()

160    0700.hk
163    0992.hk
164    2382.hk
165    0981.hk
166    2018.hk
Name: symbol, dtype: object

In [7]:
web.DataReader('0700.HK', 'yahoo', start, end).head()
# web.get_data_yahoo('0700.hk', start, end).head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,


In [10]:
hk_data.shape

(15820, 6)

In [15]:
hk_data.tail()

,High,Low,Open,Close,Volume,Adj Close,code
Date,,,,,,,
2018-12-17,1.13,1.10,1.13,1.11,5995000.0,1.11,1337.hk
2018-12-18,1.10,1.07,1.10,1.07,11671195.0,1.07,1337.hk
2018-12-19,1.08,1.06,1.07,1.07,9548819.0,1.07,1337.hk
2018-12-20,1.08,1.05,1.08,1.06,8813000.0,1.06,1337.hk
2018-12-21,1.07,1.04,1.05,1.05,9479408.0,1.05,1337.hk


In [16]:
hk_data.reset_index().set_index(['code', 'Date'])

High        Low       Open      Close      Volume  \
code    Date                                                                 
3396.hk 2015-06-29  43.549999  42.849998  43.150002  42.950001  62651090.0   
        2015-06-30  43.450001  43.000000  43.000000  43.349998   8230600.0   
        2015-07-02  43.950001  42.950001  43.349998  43.849998   7664889.0   
        2015-07-03  43.900002  42.950001  43.849998  43.000000   5044400.0   
        2015-07-06  43.500000  40.900002  43.049999  42.549999  29058194.0   
        2015-07-07  42.700001  41.750000  42.549999  42.500000  11563313.0   
        2015-07-08  41.900002  36.799999  41.799999  38.049999   6852739.0   
        2015-07-09  40.650002  38.000000  38.049999  40.299999   3573800.0   
        2015-07-10  42.250000  40.450001  40.450001  41.900002   2388400.0   
        2015-07-13  42.000000  40.849998  41.849998  41.549999   1837624.0   
        2015-07-14  41.900002  41.349998  41.549999  41.500000    389399.0   
        2015-07-15  41.500000  39.549999  41.500000  40.000000    718350.0   
        2015-07-16  39.750000  39.099998  39.500000  39.599998    921900.0   
        2015-07-17  40.500000  39.099998  39.099998  40.200001    447100.0   
        2015-07-20  41.299999  40.049999  40.950001  40.299999    973700.0   
        2015-07-21  40.500000  39.400002  40.500000  39.750000   1206800.0   
        2015-07-22  40.299999  39.549999  39.650002  39.700001    840400.0   
        2015-07-23  39.900002  39.099998  39.700001  39.150002   1513100.0   
        2015-07-24  39.500000  37.849998  39.500000  38.000000   1755700.0   
        2015-07-27  38.000000  36.049999  37.500000  36.299999   1175500.0   
        2015-07-28  36.700001  34.049999  36.299999  34.849998   2259500.0   
        2015-07-29  35.400002  34.400002  34.799999  34.849998   1135974.0   
        2015-07-30  35.099998  34.599998  35.000000  34.750000    657300.0   
        2015-07-31  35.250000  34.700001  34.700001  35.000000   1867500.0   
        2015-08-03  35.000000  33.849998  35.000000  34.000000   1300701.0   
        2015-08-04  34.650002  33.549999  34.000000  33.799999    603200.0   
        2015-08-05  34.200001  33.849998  33.849998  33.950001   1797200.0   
        2015-08-06  35.200001  34.000000  34.200001  35.099998   1005200.0   
        2015-08-07  37.000000  35.099998  35.099998  36.900002   1484411.0   
        2015-08-10  37.000000  35.799999  36.900002  36.500000    555600.0   
...                       ...        ...        ...        ...         ...   
1337.hk 2018-11-12   1.290000   1.240000   1.250000   1.250000   9219000.0   
        2018-11-13   1.260000   1.200000   1.220000   1.260000  13235000.0   
        2018-11-14   1.250000   1.210000   1.240000   1.220000  14092642.0   
        2018-11-15   1.230000   1.210000   1.220000   1.220000   5986602.0   
        2018-11-16   1.260000   1.220000   1.220000   1.250000  13998000.0   
        2018-11-19   1.260000   1.240000   1.240000   1.250000   6298623.0   
        2018-11-20   1.230000   1.210000   1.220000   1.220000   8106967.0   
        2018-11-21   1.230000   1.190000   1.190000   1.230000   8206609.0   
        2018-11-22   1.230000   1.210000   1.230000   1.210000   2573000.0   
        2018-11-23   1.210000   1.190000   1.210000   1.200000   4637000.0   
        2018-11-26   1.230000   1.190000   1.200000   1.200000   8402031.0   
        2018-11-27   1.210000   1.160000   1.210000   1.180000  13137000.0   
        2018-11-28   1.210000   1.150000   1.180000   1.200000  15837549.0   
        2018-11-29   1.230000   1.170000   1.210000   1.200000  11258963.0   
        2018-11-30   1.200000   1.180000   1.200000   1.200000   6179249.0   
        2018-12-03   1.230000   1.200000   1.200000   1.210000  11564266.0   
        2018-12-04   1.250000   1.210000   1.210000   1.230000  10347000.0   
        2018-12-05   1.230000   1.180000   1.200000   1.230000   8480222.0   
        2018-12-06   1.210000   1.180000   1.20

In [18]:
us_symbol_list = symbol_list.query("region=='US Equity'").symbol
us_symbol_list.shape

(817,)

In [24]:
us_data = pd.DataFrame()
start = '2010-01-01'
end = '2018-12-21'

for i in tqdm_notebook(us_symbol_list):
    try:
        temp_data = web.DataReader(i, 'yahoo', start, end)
        temp_data = temp_data.assign(code=i)
        us_data = pd.concat([us_data, temp_data])
    except:
        print('there is something wrong in %s' % i)

there is something wrong in IUGXX
there is something wrong in TRPXX
there is something wrong in GVMXX
there is something wrong in BRIXX
there is something wrong in HEI/A
there is something wrong in MOG/A
there is something wrong in BF/B



In [9]:
hk_data.to_csv('data/hk_data.csv')

In [23]:
# pd.read_csv('hk_test.csv', index_col=0).head()

In [25]:
us_data.to_csv('data/us_data.csv')